# 02_entiry-extract

- entity extraction from news description using trasnformer Davlan/xlm-roberta-base-ner-hrl

## install extra library

In [23]:
!pip install transformers torch
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## read stored data from DataPlatform

In [24]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd
import re

In [25]:
dir_raw_id = 62
dir_process_id = 63

In [26]:
datanode = DataNode()

Init DataNode sucessful!


In [27]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='jew_news.json'))
meta

{'owner': 'nutnicha',
 'human_size': '18.22 KB',
 'name': 'jew_news.json',
 'description': 'file jew_news.json',
 'directory': 62,
 'path': 'nutnicha/DSI310-08/rawData/jew_news.json'}

In [28]:
data = json.load(fp)
data

[{'Topic': 'Aditya Birla Group plans branded jewellery foray',
  'Describe': "The jewellery business remains largely unorganised in India. Among branded players, Titan's flagship jewellery brand Tanishq leads others by a wide margin. Other brands under Titan group's jewellery portfolio include Mia, Zoya and CaratLane. As of September this year, Titan had 488 jewellery stores in India across its portfolio of brands. Other large jewellery players like Kalyan Jewellers, Joyalukkas Group and Malabar Gold & Diamonds have all reported strong revival of demand for jewellery, amid positive consumer sentiment after two years of pandemic-induced stress.",
  'source_url': '/industry/cons-products/fashion-/-cosmetics-/-jewellery/aditya-birla-group-plans-branded-jewellery-foray/articleshow/95214892.cms',
  'Time': ' 12.13 AM ',
  'Date': '01 Nov, 2022'},
 {'Topic': 'Malabar becomes 1st Indian jeweller to import 25 kg gold from UAE under CEPA',
  'Describe': 'In May 2022, Malabar Gold and Diamonds b

# NER (NameEntitiesRegcognition)

In [29]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [30]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [31]:
example = "SafeGold introduces GAINS- a platform to lease individually owned gold"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-ORG', 'score': 0.9977634, 'index': 1, 'word': '▁Safe', 'start': 0, 'end': 4}, {'entity': 'I-ORG', 'score': 0.99686253, 'index': 2, 'word': 'Gold', 'start': 4, 'end': 8}]


In [32]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [36]:
text = data[10]['Topic']
text

'Malabar Gold & Diamonds to start construction of manufacturing unit in Telangana'

In [37]:
output = nlp(text)
output

[{'entity': 'B-ORG',
  'score': 0.9998952,
  'index': 1,
  'word': '▁Mala',
  'start': 0,
  'end': 4},
 {'entity': 'I-ORG',
  'score': 0.9994967,
  'index': 2,
  'word': 'bar',
  'start': 4,
  'end': 7},
 {'entity': 'I-ORG',
  'score': 0.9997991,
  'index': 3,
  'word': '▁Gold',
  'start': 7,
  'end': 12},
 {'entity': 'I-ORG',
  'score': 0.9998313,
  'index': 4,
  'word': '▁&',
  'start': 12,
  'end': 14},
 {'entity': 'I-ORG',
  'score': 0.9998468,
  'index': 5,
  'word': '▁Diamond',
  'start': 14,
  'end': 22},
 {'entity': 'I-ORG',
  'score': 0.9997794,
  'index': 6,
  'word': 's',
  'start': 22,
  'end': 23},
 {'entity': 'B-LOC',
  'score': 0.9997477,
  'index': 14,
  'word': '▁Telangana',
  'start': 70,
  'end': 80}]

In [38]:
post_process(output)

[{'type': 'I-ORG', 'text': ' Malabar Gold & Diamonds'},
 {'type': 'I-ORG', 'text': ' Gold & Diamonds'},
 {'type': 'I-ORG', 'text': ' & Diamonds'},
 {'type': 'I-ORG', 'text': ' Diamonds'},
 {'type': 'B-LOC', 'text': ' Telangana'}]

# Process news to spo

In [39]:
spo = []
for jew_news in tqdm(data):
    Topic = jew_news.get('Topic') 
    if Topic == None:
        continue
    ner = nlp(Topic)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'Topic': jew_news.get('Topic', ""),
            'Describe': jew_news.get('Describe', ""),
            'Date': jew_news.get('Date', ""),
            'Time': jew_news.get('Time', ""),
            'URL': jew_news.get('source_url', ""),
        })

  0%|          | 0/29 [00:00<?, ?it/s]

In [41]:
df = pd.DataFrame(spo)
df['Topic'] = df['Topic'].apply(lambda x: x[2:])
df

,Topic,Describe,Date,Time,URL
0,itya Birla Group plans branded jewellery foray,The jewellery business remains largely unorgan...,"01 Nov, 2022",12.13 AM,/industry/cons-products/fashion-/-cosmetics-/-...
1,itya Birla Group plans branded jewellery foray,The jewellery business remains largely unorgan...,"01 Nov, 2022",12.13 AM,/industry/cons-products/fashion-/-cosmetics-/-...
2,itya Birla Group plans branded jewellery foray,The jewellery business remains largely unorgan...,"01 Nov, 2022",12.13 AM,/industry/cons-products/fashion-/-cosmetics-/-...
3,labar becomes 1st Indian jeweller to import 25...,"In May 2022, Malabar Gold and Diamonds became ...","29 Oct, 2022",01.14 AM,/industry/cons-products/fashion-/-cosmetics-/-...
4,labar becomes 1st Indian jeweller to import 25...,"In May 2022, Malabar Gold and Diamonds became ...","29 Oct, 2022",01.14 AM,/industry/cons-products/fashion-/-cosmetics-/-...
5,labar becomes 1st Indian jeweller to import 25...,"In May 2022, Malabar Gold and Diamonds became ...","29 Oct, 2022",01.14 AM,/industry/cons-products/fashion-/-cosmetics-/-...
6,"ld, jewellery glitter as Dhanteras witnesses r...",Dhanteras is regarded as the most auspicious d...,"23 Oct, 2022",09.27 PM,/industry/cons-products/fashion-/-cosmetics-/-...
7,"ound Rs 40,000 cr of business expected during ...","Around Rs 40,000 crores of business is expec...","22 Oct, 2022",04.06 PM,/industry/cons-products/fashion-/-cosmetics-/-...
8,"ound Rs 40,000 cr of business expected during ...","Around Rs 40,000 crores of business is expec...","22 Oct, 2022",04.06 PM,/industry/cons-products/fashion-/-cosmetics-/-...
9,wellers expect footfalls to increase during Dh...,Jewellers are expecting the footfalls to incre...,"22 Oct, 2022",01.51 PM,/industry/cons-products/fashion-/-cosmetics-/-...


# write DataNode to DataPlatform

In [42]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='jew_news.json')])

{'sucess': True,
 'file_id': 430,
 'path': 'nutnicha/DSI310-08/processData/spo.parquet',
 'lineage': True}